In [1]:
# Libraries for data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

---

<h1><center>Data Pre-Processing

---

In [2]:
# Download data
df = pd.read_csv('ev_battery_charging_data.csv')

In [3]:
# Drop rows with any NaN values
df_clean = df.dropna()

# Shape after cleaning
print("\nAfter removing rows with NaN:")
print(f"Rows: {df_clean.shape[0]}, Columns: {df_clean.shape[1]}")


After removing rows with NaN:
Rows: 1000, Columns: 13


In [4]:
# Drop inputs calculated from the column we are predicting
drop_cols = ['Degradation Rate (%)', 'Efficiency (%)', 'Optimal Charging Duration Class']
df_clean = df_clean.drop(columns=drop_cols)

In [5]:
# List the categorical columns and numerical columns for one-hot encoding later
cat_col = ['Charging Mode', 'Battery Type', 'EV Model']
num_col = ['SOC (%)', 'Voltage (V)', 'Current (A)', 'Battery Temp (°C)', 'Ambient Temp (°C)', 'Charging Cycles']
target = 'Charging Duration (min)'

In [6]:
# Move the column we are predicting to the end
df_clean[target] = df_clean.pop(target)

In [7]:
# Split the data into test and train
from sklearn.model_selection import train_test_split
X = df_clean.drop(columns=target)
y = df_clean[target]
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)

In [8]:
df_clean.head()

,SOC (%),Voltage (V),Current (A),Battery Temp (°C),Ambient Temp (°C),Charging Mode,Battery Type,Charging Cycles,EV Model,Charging Duration (min)
0,43.708611,3.629593,33.553512,33.454060,26.439918,Fast,Li-ion,112,Model B,59.363552
1,95.564288,3.879331,32.228092,35.933628,31.108647,Fast,LiFePO4,398,Model A,67.343566
2,75.879455,4.111062,91.562912,25.009358,30.203219,Slow,LiFePO4,175,Model A,105.454739
3,63.879264,4.012557,32.459158,32.497482,18.077998,Fast,LiFePO4,150,Model B,54.000439
4,24.041678,4.064593,34.475475,31.434920,17.984989,Fast,Li-ion,886,Model C,106.964968


In [9]:
df_clean.describe(include='all')

,SOC (%),Voltage (V),Current (A),Battery Temp (°C),Ambient Temp (°C),Charging Mode,Battery Type,Charging Cycles,EV Model,Charging Duration (min)
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000,1000,1000.000000,1000,1000.000000
unique,NaN,NaN,NaN,NaN,NaN,3,2,NaN,3,NaN
top,NaN,NaN,NaN,NaN,NaN,Slow,LiFePO4,NaN,Model C,NaN
freq,NaN,NaN,NaN,NaN,NaN,341,503,NaN,343,NaN
mean,54.123090,3.854912,55.216515,29.807501,24.882106,NaN,NaN,556.560000,NaN,69.846449
std,26.292363,0.204533,26.160678,5.729787,5.736199,NaN,NaN,263.763052,NaN,28.919481
min,10.416882,3.502253,10.001047,20.013068,15.000614,NaN,NaN,101.000000,NaN,20.618382
25%,31.237594,3.668752,33.521588,24.838015,19.899480,NaN,NaN,317.750000,NaN,44.938000
50%,54.712664,3.863114,55.055253,29.685586,24.891962,NaN,NaN,571.000000,NaN,69.040282
75%,76.988763,4.032326,78.319317,34.750816,29.799869,NaN,NaN,786.000000,NaN,93.985178


---

<h1><center>Linear Regression Models

---

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

In [11]:
# Preprocessor for numerical columns
num_transformer = Pipeline(
    steps=[("scaler", StandardScaler())])

# Preprocessor for categorical columns
cat_transformer = Pipeline(
    steps=[("encoder", OneHotEncoder(handle_unknown="ignore"))])

# Combine preprocessors
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_col),
        ('cat', cat_transformer, cat_col)])

In [12]:
# Linear regression with feature selection
linreg_pipe = Pipeline([('preprocessor', preprocessor), 
                     ('linreg', LinearRegression())])

# Fit the model
linreg_pipe.fit(Xtrain, ytrain)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['SOC (%)', 'Voltage (V)',
                                                   'Current (A)',
                                                   'Battery Temp (°C)',
                                                   'Ambient Temp (°C)',
                                                   'Charging Cycles']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Charging Mode',
                                                   'Battery Type',
                                                   'EV Model'])])),
                ('linreg', LinearRegression())])

In [13]:
# Lasso regression
alphas = np.logspace(-4, 4, 10)
lasso_models = []

for alpha in alphas:
    lasso_pipe = Pipeline([('preprocessor', preprocessor), 
                         ('lasso', Lasso(alpha=alpha))])
    lasso_models.append(lasso_pipe)

In [14]:
pred = linreg_pipe.predict(Xtest)
r2 = r2_score(ytest, pred)

<h2><center>Logistic Regression Comparison

In [15]:
from sklearn.linear_model import LogisticRegression

---

<h1><center>Neural Networks

---

---

<h1><center>Random Forests

---